Reviewing Additive Secret Sharing

In [2]:
import random
import numpy as np

BASE = 10

PRECISION_INTEGRAL = 8
PRECISION_FRACTIONAL = 8
Q = 293973345475167247070445277780365744413

PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL

assert(Q > BASE**PRECISION)

def encode(rational):
    upscaled = int(rational * BASE ** PRECISION_FRACTIONAL)
    field_element = upscaled % Q
    return field_element

def decode(field_element):
    upscaled = field_element if field_element <= Q/2 else field_element - Q
    rational = upscaled / BASE**PRECISION_FRACTIONAL
    return rational

def encrypt(secret):
    first = random.randrange(Q)
    second = random.randrange(Q)
    third = (secret - first - second) % Q
    return [first, second, third]

def decrypt(sharing):
    return sum(sharing) % Q

def add(a,b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [4]:
x = encrypt(encode(5.5))
x

[161353318393223126974070492716088813281,
 78970506255405229314004598725267533684,
 53649520826538890782370186339559397448]

In [6]:
y = encrypt(encode(2.3))
y

[103270652661025053799666998193493009981,
 176698969839536808305497334372266577130,
 14003722974605384965280945214836157301]

In [7]:
z = add(x,y)
z

(264623971054248180773737490909581823262,
 255669476094942037619501933097534110814,
 67653243801144275747651131554395554749)

In [8]:
decode(decrypt(z))

7.79999999

Encrypted Subtraction and Public/Scalar Multiplication

In [9]:
field = 23740629843760239486723

In [12]:
x = 5

bob_x_share = 2372385723 #random number
alices_x_share = field - bob_x_share + x

In [13]:
(bob_x_share + alices_x_share) % field

5

In [14]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 1

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [15]:
((bob_x_share + alice_x_share) - (bob_y_share + alice_y_share)) % field

4

In [16]:
((bob_x_share - bob_y_share) + (alice_x_share - alice_y_share)) % field

4

In [17]:
bob_x_share + alice_x_share + bob_y_share + alice_y_share

26

In [18]:
bob_z_share = (bob_x_share - bob_y_share)
alice_z_share = (alice_x_share - alice_y_share)

In [19]:
(bob_z_share + alice_z_share) % field

4

In [20]:
def sub(a,b):
    c=list()
    for i in range(len(a)):
        c.append((a[i] - b[i]) % Q)
    return tuple(c)

In [21]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 1

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [22]:
bob_x_share + alice_x_share

15

In [23]:
bob_y_share + alice_y_share

11

In [24]:
((bob_y_share * 3) + (alice_y_share * 3)) % field

3

In [25]:
def imul(a,scalar):
    
    #logic here which can multiply by a public scalar
    
    c = list()
    
    for i in range(len(a)):
        c.append((a[i] * scalar) % Q)
        
    return tuple(c)

In [26]:
x = encrypt(encode(5.5))
x

[266973104040500757282494160859001233731,
 263505042295251858881935816464817267171,
 57468544614581877976460578237462987924]

In [27]:
z = imul(x,3)

In [28]:
decode(decrypt(z))

16.5

Encrypted Computation in PySyft

In [29]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn,optim

W0809 13:03:56.980319  6264 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'C:\Users\Vilas_2\Anaconda3\envs\pysyft\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0809 13:03:57.244334  6264 deprecation_wrapper.py:119] From C:\Users\Vilas_2\Anaconda3\envs\pysyft\lib\site-packages\tf_encrypted\session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [30]:
bob = sy.VirtualWorker(hook, id='bob').add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id='alice').add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id='secure_worker').add_worker(sy.local_worker)

In [31]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

In [32]:
x = x.share(bob, alice, crypto_provider = secure_worker)

In [33]:
y = y.share(bob, alice, crypto_provider = secure_worker)

In [34]:
z = x+y
z.get()

tensor([3, 1, 4, 4])

In [35]:
z = x-y
z.get()

tensor([-1,  3,  2,  4])

In [36]:
z = x * y
z.get()

tensor([ 2, -2,  3,  0])

In [37]:
z = x > y
z.get()

tensor([0, 1, 1, 1])

In [38]:
z = x < y
z.get()

tensor([1, 0, 0, 0])

In [39]:
z = x == y
z.get()

tensor([0, 0, 0, 0])

In [40]:
x = th.tensor([1,2,3,4])
y= th.tensor([2,-1,1,0])

x = x.fix_precision().share(bob, alice, crypto_provider=secure_worker)
y = y.fix_precision().share(bob, alice, crypto_provider=secure_worker)

In [41]:
z = x+y
z.get().float_precision()

tensor([3., 1., 4., 4.])

In [42]:
z = x-y
z.get().float_precision()

tensor([-1.,  3.,  2.,  4.])

In [43]:
z = x*y
z.get().float_precision()

tensor([ 2., -2.,  3.,  0.])

In [44]:
z = x>y
z.get().float_precision()

tensor([0., 1., 1., 1.])

In [45]:
z = x<y
z.get().float_precision()

tensor([1., 0., 0., 0.])

In [46]:
z = x==y
z.get().float_precision()

tensor([0., 0., 0., 0.])

Build an Encrypted Database

In [47]:
import string

In [48]:
char2index = {}
index2char ={}

In [56]:
for i, char in enumerate(' ' + string.ascii_lowercase + '0123456789' + string.punctuation):
    char2index[char] = i
    index2char[i] = char

In [57]:
str_input = 'Hello'
max_len = 8

In [58]:
def string2values(str_input, max_len=8):
    
    str_input = str_input[:max_len].lower()
    
    #pad strings shorter than max len
    if(len(str_input) < max_len):
        str_input = str_input + '.' * (max_len - len(str_input))
        
    values = list()
    for char in str_input:
        values.append(char2index[char])
        
    return th.tensor(values).long()

def values2string(input_values):
    s = ""
    for value in input_values:
        s += index2char[int(value)]
    return s

def strings_equal(str_a, str_b):
    vect = (str_a * str_b).sum(1)
    x = vect[0]
    for i in range(vect.shape[0] - 1):
        x = x * vect[i + 1]
        
    return x

def one_hot(index, length):
    vect = th.zeros(length).long()
    vect[index] = 1
    return vect

def string2one_hot_matrix(str_input, max_len=8):
    str_input = str_input[:max_len].lower()
    
    #pad strings shorter than max len
    if(len(str_input) < max_len):
        str_input = str_input + '.' * (max_len - len(str_input))
        
    char_vectors = list()
    for char in str_input:
        char_v = one_hot(char2index[char], len(char2index)).unsqueeze(0)
        char_vectors.append(char_v)
        
    return th.cat(char_vectors, dim=0)

In [61]:
class EncryptedDB():
    def __init__(self, *owners, max_key_len=8, max_val_len=8):
        self.max_key_len = max_key_len
        self.max_val_len = max_val_len
        
        self.keys = list()
        self.values = list()
        self.owners = owners
        
    def add_entry(self, key, value):
        key = string2one_hot_matrix(key)
        key = key.share(*self.owners)
        self.keys.append(key)
        
        value = string2values(value, max_len=self.max_val_len)
        value = value.share(*self.owners)
        self.values.append(value)
    
    def query(self, query_str):
        query_matrix = string2one_hot_matrix(query_str)
        
        query_matrix = query_matrix.share(*self.owners)
        
        key_matches = list()
        for key in self.keys:
            
            key_match = strings_equal(key, query_matrix)
            key_matches.append(key_match)
            
        result = self.values[0] * key_matches[0]
        
        for i in range(len(self.values) - 1):
            result += self.values[i+1] * key_matches[i+1]
            
        result = result.get()
        
        return values2string(result).replace(".","")

In [62]:
db = EncryptedDB(bob, alice, secure_worker, max_val_len=256)

db.add_entry("Bob", "(123) 456 789")
db.add_entry("Bill", "(234) 567 8901")
db.add_entry("Sam", "(345) 678 9012")
db.add_entry("Key", "really big json value")

db.query("Bob")

'(123) 456 789'

Encrypted Data Learning in PyTorch

In [64]:
#Train a Model
from torch import nn
from torch import optim
import torch.nn.functional as F

In [66]:
# A toy dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(2,20)
        self.fc2 = nn.Linear(20,1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x
    
# A Toy Model
model = Net()

def train():
    #Training logic
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    for iter in range(20):
        
        #1. clear previous gradient (if they exist)
        opt.zero_grad()
        
        #2. make a prediction
        pred = model(data)
        
        #3. calculate loss
        loss = ((pred - target)**2).sum()
        
        #4. figure out which weights caused us to miss
        loss.backward()
        
        #5. change those weights
        opt.step()
        
        #6. print out progress
        print(loss.data)

train()

tensor(2.1447)
tensor(7.9090)
tensor(21.8189)
tensor(3.1027)
tensor(0.9762)
tensor(0.8614)
tensor(0.7832)
tensor(0.6830)
tensor(0.5701)
tensor(0.4644)
tensor(0.3602)
tensor(0.2658)
tensor(0.1870)
tensor(0.1253)
tensor(0.0812)
tensor(0.0516)
tensor(0.0326)
tensor(0.0208)
tensor(0.0135)
tensor(0.0089)


In [67]:
model(data)

tensor([[0.0670],
        [0.0149],
        [1.0047],
        [0.9640]], grad_fn=<AddmmBackward>)

Encrypt the Model and Data

In [69]:
encrypted_model = model.fix_precision().share(alice,bob,crypto_provider=secure_worker)

In [70]:
list(encrypted_model.parameters())

[Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:16882532305 -> alice:8012152303]
 	-> (Wrapper)>[PointerTensor | me:44401383409 -> bob:63734484058]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:80239947319 -> alice:9848748363]
 	-> (Wrapper)>[PointerTensor | me:70798805584 -> bob:44628495639]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:62749576212 -> alice:77039444226]
 	-> (Wrapper)>[PointerTensor | me:92669072916 -> bob:82960716361]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:14634130139 -> alice:10140916021]
 	-> (Wrapper)>[PointerTensor | me:54585943398 -> bob:5054751361

In [71]:
encrypted_data = data.fix_precision().share(alice,bob,crypto_provider=secure_worker)

In [72]:
encrypted_data

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:53077325727 -> alice:75057937839]
	-> (Wrapper)>[PointerTensor | me:56257978128 -> bob:23312242144]
	*crypto provider: secure_worker*

In [73]:
encrypted_prediction = encrypted_model(encrypted_data)

In [74]:
encrypted_prediction.get().float_precision()

tensor([[0.0670],
        [0.0150],
        [1.0030],
        [0.9630]])